# Setup

## Make both T4 GPUs visiable to CUDA

In [1]:
import os, math, numpy as np
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

## Install vLLM

In [6]:
%%time
!pip uninstall -y torch
!pip install -U --no-index --find-links=/kaggle/input/vllm-whl -U vllm
!pip install -U --upgrade /kaggle/input/vllm-t4-fix/grpcio-1.62.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install -U --upgrade /kaggle/input/vllm-t4-fix/ray-2.11.0-cp310-cp310-manylinux2014_x86_64.whl

Found existing installation: torch 2.1.2
Uninstalling torch-2.1.2:
  Successfully uninstalled torch-2.1.2
Looking in links: /kaggle/input/vllm-whl
Processing /kaggle/input/vllm-whl/vllm-0.4.0.post1-cp310-cp310-manylinux1_x86_64.whl
Processing /kaggle/input/vllm-whl/torch-2.1.2-cp310-cp310-manylinux1_x86_64.whl (from vllm)
Processing /kaggle/input/vllm-whl/xformers-0.0.23.post1-cp310-cp310-manylinux2014_x86_64.whl (from vllm)
Processing /kaggle/input/vllm-whl/outlines-0.0.34-py3-none-any.whl (from vllm)
Processing /kaggle/input/vllm-t4-fix/grpcio-1.62.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
grpcio is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing /kaggle/input/vllm-t4-fix/ray-2.11.0-cp310-cp310-manylinux2014_x86_64.whl
ray is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
CPU times: user 767 ms, sys: 193

## Some Configuration

In [35]:
# In DEBUG mode, infer only on 5 problems
DEBUG = False
# Number of candidate solutions to generate
K = 7
DEPTH = 4
TEMPERATURE = 0.75
TOP_P = 0.85
BATCH_SIZE = 64

## Imports

In [8]:
import vllm
import re
import csv
import torch
import gc
from tqdm import tqdm
import pandas as pd
from queue import Queue, Empty
import os
import re
import signal
import subprocess
import tempfile
from collections import Counter
from contextlib import contextmanager

import threading
from concurrent.futures import ThreadPoolExecutor, as_completed

2024-10-20 16:10:20,550	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


# Python Code Execution Environment

In [9]:
1

1

## Find Python code blocks within text

In [14]:
def find_python_blocks(text):
    blocks = re.findall(r"(```python.*?```)", text, re.DOTALL)
    # filter blocks by trying to convert them to float or int
    filtered_blocks = []
    for block in blocks:
        code = block[len("```python"):-len("```")].strip()
        try:
            x = int(code)
        except:
            filtered_blocks.append(code)
            continue
        try:
            x = float(code)
        except:
            filtered_blocks.append(code)
    return filtered_blocks        

## Class to Execute Python code (adopted from Numina)

In [15]:
class PythonREPL:
    def __init__(self, timeout=5):
        self.timeout = timeout
    # handles timeout
    @contextmanager
    def time_limit(self, seconds):
        def signal_handler(*_):
            raise TimeoutError(f"Timed out after {seconds} seconds.")

        signal.signal(signal.SIGALRM, signal_handler)
        signal.alarm(seconds)
        try:
            yield
        finally:
            signal.alarm(0)

    def __call__(self, query):
        query = "import math\nimport numpy as np\nimport sympy as sp\n" + query
        query = query.strip().split("\n")
        if "print(" not in query[-1]:
            if "#" in query[-1]:
                query[-1] = query[-1].split("#")[0]
            query[-1] = "print(" + query[-1] + ")"
        query = "\n".join(query)
        with tempfile.TemporaryDirectory() as temp_dir:
            temp_file_path = os.path.join(temp_dir, "tmp.py")
            with open(temp_file_path, "w", encoding="utf-8") as f:
                f.write(query)
            with self.time_limit(self.timeout):
                result = subprocess.run(
                    ["python3", temp_file_path],
                    capture_output=True,
                    check=False,
                    text=True,
                    timeout=self.timeout,
                )
                if result.returncode == 0:
                    output = result.stdout
                    return True, output.strip()
                error_msg = result.stderr.strip()
                msgs = error_msg.split("\n")
                new_msgs = []
                want_next = False
                for m in msgs:
                    if "Traceback" in m:
                        new_msgs.append(m)
                    elif m == msgs[-1]:
                        new_msgs.append(m)
                    elif temp_file_path in m:
                        st = m.index('"/') + 1 if '"/' in m else 0
                        ed = m.index(temp_file_path) + 1 if temp_file_path in m else None
                        clr = m[st:ed] if not ed else m[st:]
                        m = m.replace(clr, "")
                        new_msgs.append(m)
                        want_next = True
                    elif want_next:
                        new_msgs.append(m)
                        want_next = False
                error_msg = "\n".join(new_msgs)
                return False, error_msg.strip()

## Execute a Python code block

In [16]:
def execute(executor, code):
    success = False
    for lib in ("subprocess", "venv"):
        if lib in code:
            output = f"{lib} is not allowed"
            outputs.append(output)
            successes.append(success)
            continue
    try:
        success, output = executor(code)
    except TimeoutError as e:
        output = str(e)

    output = output.strip()
    
    return output, success

## Test by running some python code

In [17]:
text = """Block 1
```python
s = 0
for i in range(100):
    s += i
print(s)
```
Block 2
```python
2**12
```
Block 3
```python
3
```
"""

In [18]:
blocks = find_python_blocks(text)
blocks

['s = 0\nfor i in range(100):\n    s += i\nprint(s)', '2**12']

In [19]:
executor = PythonREPL()
outputs = [execute(executor, block) for block in blocks]
outputs

[('4950', True), ('4096', True)]

In [20]:
for block, output in zip(blocks, outputs):
    print(f"""
```python
{block}
```
```output
{output[0]}
```"""
    )


```python
s = 0
for i in range(100):
    s += i
print(s)
```
```output
4950
```

```python
2**12
```
```output
4096
```


# Load Model on vLLM

## We use the Qwen 2.5 7b Instruct Model here by Alibaba. You should explore other models.

In [21]:
llm = vllm.LLM(
    "AI-MO/NuminaMath-7B-TIR",
    tensor_parallel_size=2, 
    gpu_memory_utilization=0.95, 
    trust_remote_code=True,
    dtype="half", 
    enforce_eager=True,
    max_model_len=4096,    
)
tokenizer = llm.get_tokenizer()

config.json:   0%|          | 0.00/716 [00:00<?, ?B/s]

WARNING 10-20 16:11:28 config.py:767] Casting torch.bfloat16 to torch.float16.


2024-10-20 16:11:31,973	INFO worker.py:1749 -- Started a local Ray instance.


INFO 10-20 16:11:33 llm_engine.py:74] Initializing an LLM engine (v0.4.0.post1) with config: model='AI-MO/NuminaMath-7B-TIR', tokenizer='AI-MO/NuminaMath-7B-TIR', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=2, disable_custom_all_reduce=True, quantization=None, enforce_eager=True, kv_cache_dtype=auto, device_config=cuda, seed=0)


tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

...

INFO 10-20 16:11:42 selector.py:40] Cannot use FlashAttention backend for Volta and Turing GPUs.
INFO 10-20 16:11:42 selector.py:25] Using XFormers backend.
(RayWorkerVllm pid=421) INFO 10-20 16:11:43 selector.py:40] Cannot use FlashAttention backend for Volta and Turing GPUs.
(RayWorkerVllm pid=421) INFO 10-20 16:11:43 selector.py:25] Using XFormers backend.
(RayWorkerVllm pid=421) INFO 10-20 16:11:44 pynccl_utils.py:45] vLLM is using nccl==2.18.1
INFO 10-20 16:11:44 pynccl_utils.py:45] vLLM is using nccl==2.18.1
INFO 10-20 16:11:45 weight_utils.py:177] Using model weights format ['*.safetensors']
(RayWorkerVllm pid=421) INFO 10-20 16:11:45 weight_utils.py:177] Using model weights format ['*.safetensors']


model-00003-of-00003.safetensors:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

INFO 10-20 16:12:49 model_runner.py:104] Loading model weights took 6.4663 GB
(RayWorkerVllm pid=421) INFO 10-20 16:12:50 model_runner.py:104] Loading model weights took 6.4663 GB
INFO 10-20 16:12:53 ray_gpu_executor.py:240] # GPU blocks: 1755, # CPU blocks: 1092


# Utilites

## Extract boxed answer

In [22]:
def extract_answer(text):
    # find right most boxed answer
    def last_boxed_only_string(text):
        idx = text.rfind("\\boxed")
        if idx < 0:
            idx = text.rfind("\\fbox")
            if idx < 0:
                return None
        i = idx
        right_brace_idx = None
        num_left_braces_open = 0
        while i < len(text):
            if text[i] == "{":
                num_left_braces_open += 1
            if text[i] == "}":
                num_left_braces_open -= 1
                if num_left_braces_open == 0:
                    right_brace_idx = i
                    break
            i += 1
        if right_brace_idx is None:
            return None
        return text[idx : right_brace_idx + 1]
    # get content of boxed
    def remove_boxed(boxed):
        left = "\\boxed{"
        try:
            assert boxed[: len(left)] == left
            assert boxed[-1] == "}"
            length = len(left)
            return boxed[length:-1]
        except Exception:
            return None

    boxed = last_boxed_only_string(text)
    if boxed is None:
        return None
    answer = remove_boxed(boxed)
    return answer

## Majority vote (select the most occuring answer)

In [23]:
# Define the majority voting function to get the most common answer
def majority_vote(answers):
    answers = [answer for answer in answers if answer is not None]

    if not answers:
        return None
    # count the occurence of each answer
    counts = {}
    for answer in answers:
        if answer in counts:
            counts[answer] += 1
        else:
            counts[answer] = 1

    max_answer = None
    max_count = 0
    
    for answer, count in counts.items():
        if count > max_count:
            max_answer = answer
            max_count = count
    
    return max_answer

# TIR Agent

In [25]:
class TIRAgent:
    def __init__(self, problem_id, id, problem, tokenizer, max_depth, log):
        # problem id
        self.problem_id = problem_id
        # id of the agent
        self.id = id
        # number of LLM turns
        self.depth = 1
        # maximum number of turns allowed
        self.max_depth = max_depth
        # LLM's tokenizer
        self.tokenizer = tokenizer
        # Problem statement
        self.problem = problem
        # Chat Messages
        self.messages = [
            {
                "role": "user", 
                "content": f"""Here is a math problem in Bengali:
{self.problem}
The answer is a non-negative integer. Please reason step by step to solve the problem above. Provide python code to verify your reasoning.
Put your final integer answer within \\boxed{{}}."""
            }
        ]
        # Last response from the LLM
        self.last_response = None
        # Code blocks from the last response
        self.blocks = []
        # Answers that the LLM generated in \boxed{}
        self.answers = []
        # No python code generated in last response or max_depth reached
        self.is_complete = False
        # File to log answers
        self.log = log
        # Next prompt to the LLM
        self.next_prompt = None
        
    def complete(self):
        # is the Agent done
        return self.is_complete
    
    def add_response(self, response, executor):
        self.depth += 1
        # remember this response
        self.last_response = response
        # add this to the messages history
        self.messages.append({"role": "assistant", "content": response})
        # extract python blocks
        self.blocks = find_python_blocks(response)
        # extract answer from the generated text, if present
        answer = extract_answer(response)
        if answer is not None:
            self.answers.append(answer)
        # is it done?
        self.is_complete = not self._should_continue()
        # if not, use the python executor to create next prompt
        if not self.is_complete:
            self.next_prompt = self._next_prompt(executor)   
            self.messages.append({"role": "user", "content": self.next_prompt})
    
    def _should_continue(self):        
        # quit if max_depth number of turns reached
        if self.depth >= self.max_depth:
            return False
        # if no python code generated, we can stop now
        elif len(self.blocks) > 0:
            return True
        return False
    
    def _next_prompt(self, executor):
        assert not self.is_complete
        assert len(self.blocks) > 0
        # get code result from python execution
        output, status = execute(executor, self.blocks[-1])
        
        prompt = ''
        # if code succeeds give the output
        if status:
            prompt = f"""The python code you provided gives the following output:
```python
{self.blocks[-1]}
```
```output
{output}
```"""
        # if code fails, give the error
        else:
            prompt = f"""The python code you provided gives the following error:
```python
{self.blocks[-1]}
```
```output
{output}
```"""
        return prompt
    
    
    def next_message(self):
        assert not self.is_complete 
        # apply chat template to get the text
        text = self.tokenizer.apply_chat_template(
            self.messages,
            tokenize=False,
            add_generation_prompt=True
        )
        
        return text
        
    
    def final_answer(self):
        # if there no answers yet, we have to return None
        ans = None
        # otherwise return the latest answer
        if len(self.answers) > 0:
            ans = self.answers[-1]
        # log to file
        if self.log:
            self.log.writerow([self.problem_id, self.id, ans])
        # try to convert to integer
        try:
            ans = int(ans)
        except:
            ans = None
        
        return ans        

# Sc-TIR Agent

In [26]:
class SCTIRAgent:
    def __init__(self, problem_id, problem, tokenizer, samples, max_depth, log):
        # problem id
        self.problem_id = problem_id
        # problem statement
        self.problem = problem
        # LLM's tokenizer
        self.tokenizer = tokenizer
        # number of TIRAgents to create
        self.samples = samples
        # maximum number of turns
        self.max_depth = max_depth
        # TIR Agents
        self.agents = [TIRAgent(problem_id, i, problem, tokenizer, max_depth, log) for i in range(samples)]
        # log file
        self.log = log
    
    def complete(self):
        # only complete when all agents are done
        for agent in self.agents:
            if not agent.complete():
                return False
        return True
        
    def get_ready_agents(self):
        # return agents that are not complete yet
        ready_agents = []
        for agent in self.agents:
            if not agent.complete():
                ready_agents.append(agent)
        return ready_agents
    
    def final_answer(self):
        # majority vote agent answers
        assert self.complete()
        answers = [agent.final_answer() for agent in self.agents]
        answer = majority_vote(answers)
        if answer is None:
            return 0
        return answer

# Load Test Set

In [27]:
test_df = pd.read_csv('/kaggle/input/dlsprint3/test.csv')
test_df

,ID,Problem
0,0,একটি কেক-কে সরলরৈখিকভাবে 2 বার কেটে সর্বোচ্চ ক...
1,1,একটি পুকুরের উপর 100 টি পাথর রাখা আছে। প্রথমে ...
2,2,"ধরো $f(x) = x^{67-x^{67-x^{67-\dots}}}$, যেখান..."
3,3,সামিন ও স্বর্গ গণনার জন্য শুধু 0 আর 1 ব্যবহার ...
4,4,"A, B, C তিনটি স্ট্যান্ড। A স্ট্যান্ড থেকে 2 টি..."
...,...,...
95,95,একটি বক্সে কিছু লাল এবং নীল বল রয়েছে। যদি আরও ...
96,96,একটি চার অঙ্কের পূর্ণসংখ্যা 11 দ্বারা নিঃশেষে ...
97,97,এমন কতগুলো ধনাত্মক পূর্ণসংখ্যা আছে যারা $12^{1...
98,98,"3 টি ক্রমিক পূর্ণসংখ্যার যোগফল 216 হলে, সবচেয়..."


In [21]:
# import pandas as pd

# translated_df = pd.read_csv('/kaggle/input/translated/translation.csv')

In [22]:
# for index, row in translated_df.iterrows():
#     cleaned_problem = row['Problem'].split("English Statement:", 1)[0].rstrip()
#     translated_df.at[index, 'Problem'] = cleaned_problem
    
#     print(f"i{index}", row['Problem'], '\n')


In [23]:
# test_df = translated_df

## Load 5 problems since we are short on time

In [28]:
if DEBUG:
    test_df = test_df
    torch.cuda.empty_cache()
    gc.collect()

# Configure LLM and Python REPL

In [36]:
sampling_params = vllm.SamplingParams(max_tokens=3582, temperature=TEMPERATURE, top_p=TOP_P)
executor = PythonREPL()

# Run the Agents

## TIR Agent

In [ ]:
# for row in test_df.values[:2]:
#     problem_id = row[0]
#     problem = row[1]
    
#     agent = TIRAgent(problem_id, 0, problem, tokenizer, max_depth=4, log=None)
    
#     while not agent.complete():
#         text = agent.next_message()
#         # get response from LLM
#         response = llm.generate([text], sampling_params)
#         # pass in python executor, since response might contain python code
#         agent.add_response(response[0].outputs[0].text, executor)
    
#     for message in agent.messages:
#         print(f"Role: {message['role']}\n")
#         print(f"Content:\n {message['content']}\n")
    
#     answer = agent.final_answer()
#     print(f"Final answer: {answer}")
    

## SC-TIR Agent

In [ ]:
# for row in test_df.values:
#     problem_id = row[0]
#     problem = row[1]
    
#     agent = SCTIRAgent(problem_id, problem, tokenizer, samples=2, max_depth=4, log=None)
    
#     while not agent.complete():
#         ready_agents = agent.get_ready_agents()
#         texts = [a.next_message() for a in ready_agents]
#         # get response from LLM
#         responses = llm.generate(texts, sampling_params)
#         # pass response to the agents
#         for i, ready_agent in enumerate(ready_agents):
#             ready_agent.add_response(responses[i].outputs[0].text, executor)
    
#     answer = agent.final_answer()
#     print(f"Problem: {problem}")
#     print(f"Final answer: {answer}")

# Run Inference

# Create submission

## Also log agent answers

In [37]:
file = open('submission.csv', 'w', encoding='utf-8')
log_file = open('log.csv', 'w', encoding='utf-8')

submission = csv.writer(file)
log = csv.writer(log_file)

submission.writerow(['ID', 'Answer'])
log.writerow(['ID', "Agent ID", 'Answer'])

20

## Configure LLM sampling parameters and Python REPL

## Use a queue to Batch inference

In [38]:
%%time

boxed_answers = {}
agents = []

q = Queue()

iterator = iter(tqdm(test_df.values))

while True:
    for agent in agents:
        if agent.complete():
            boxed_answers[agent.problem_id] = agent.final_answer()
            print(agent.final_answer())

    agents[:] = list(filter(lambda a: not a.complete(), agents))

    while q.qsize() < BATCH_SIZE:
        try:
            row = next(iterator)
        except StopIteration:
            break

        id = row[0]
        problem = row[1]

        agent = SCTIRAgent(id, problem, tokenizer, K, DEPTH, log)
        
        agents.append(agent)

        for tir_agent in agent.get_ready_agents():
            q.put_nowait(tir_agent)
            
    if q.empty():
        break
        
    
    ready_agents = []
    texts = []
    for _ in range(BATCH_SIZE):
        try:
            agent = q.get_nowait()
            ready_agents.append(agent)
            texts.append(agent.next_message())
        except:
            break

    
    responses = llm.generate(texts, sampling_params)
    responses = [response.outputs[0].text for response in responses]
    
    for i in range(len(ready_agents)):
        agent = ready_agents[i]
        response = responses[i]
        agent.add_response(response, executor)
        if not agent.complete():
            q.put_nowait(agent)
   

Processed prompts: 100%|██████████| 64/64 [02:32<00:00,  2.39s/it]

Processed prompts:   0%|          | 0/64 [00:00<?, ?it/s]

WARNING 10-20 19:35:59 scheduler.py:245] Input prompt (4249 tokens) is too long and exceeds limit of 4096



Processed prompts:  45%|████▌     | 29/64 [03:01<06:13, 10.68s/it]

WARNING 10-20 19:38:54 scheduler.py:245] Input prompt (11726 tokens) is too long and exceeds limit of 4096



 14%|█▍        | 14/100 [13:19<1:26:20, 60.24s/it]

...

WARNING 10-20 21:49:18 scheduler.py:245] Input prompt (4326 tokens) is too long and exceeds limit of 4096


Processed prompts: 100%|██████████| 28/28 [02:05<00:00,  4.47s/it]


1012
9
93192


Processed prompts: 100%|██████████| 6/6 [02:35<00:00, 25.91s/it]

5757
71
4
CPU times: user 2h 12min 49s, sys: 34.3 s, total: 2h 13min 24s
Wall time: 2h 25min 44s


## Write to submission file

In [39]:
sorted_boxed_answers = {id: abs(answer) for id, answer in sorted(boxed_answers.items())}

# Write the sorted and modified answers to your CSV
for id, answer in sorted_boxed_answers.items():
    submission.writerow([id, answer])

In [30]:
for id, answer in boxed_answers.items():
    submission.writerow([id, answer])

In [40]:
sorted_boxed_answers

{0: 3,
 1: 10,
 2: 3,
 3: 225,
 4: 40,
 5: 0,
 6: 120,
 7: 334,
 8: 14,
 9: 63,
 10: 2,
 11: 8,
 12: 1901,
 13: 1000,
 14: 16000,
 15: 114000,
 16: 11,
 17: 8,
 18: 17,
 19: 24,
 20: 97,
 21: 96,
 22: 15,
 23: 1,
 24: 145,
 25: 4105,
 26: 0,
 27: 3,
 28: 29,
 29: 1,
 30: 201,
 31: 280,
 32: 17,
 33: 1224,
 34: 2,
 35: 12,
 36: 0,
 37: 50,
 38: 252,
 39: 9,
 40: 102,
 41: 8,
 42: 20,
 43: 4,
 44: 857142,
 45: 65,
 46: 2,
 47: 75000,
 48: 7,
 49: 525,
 50: 44,
 51: 53,
 52: 5,
 53: 36,
 54: 11,
 55: 7,
 56: 21,
 57: 0,
 58: 4,
 59: 12,
 60: 1083,
 61: 10,
 62: 22335577,
 63: 15,
 64: 1349,
 65: 0,
 66: 999,
 67: 465,
 68: 11,
 69: 9,
 70: 16,
 71: 3130,
 72: 16,
 73: 529,
 74: 16,
 75: 11,
 76: 120,
 77: 6,
 78: 57,
 79: 62281,
 80: 110,
 81: 45620,
 82: 0,
 83: 128,
 84: 24,
 85: 5,
 86: 20,
 87: 25,
 88: 132,
 89: 105,
 90: 28,
 91: 169,
 92: 3,
 93: 24,
 94: 1012,
 95: 9,
 96: 93192,
 97: 5757,
 98: 71,
 99: 4}

## Close files

In [41]:
   
file.close()
log_file.close()